## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
sys.path.append("..")
from config import g_key


#from config_file import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [33]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Rikitea,PF,-23.1203,-134.9692,81.21,79,37,8.99,light rain
1,1,Dingle,PH,10.9995,122.6711,84.90,65,41,15.64,scattered clouds
2,2,Puerto Ayora,EC,-0.7393,-90.3518,77.00,92,24,3.00,few clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,55.40,71,75,12.66,broken clouds
4,4,Okhotsk,RU,59.3833,143.3000,-0.13,80,0,6.29,clear sky


In [34]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [35]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [37]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                187
City                   187
Country                187
Lat                    187
Lng                    187
Max Temp               187
Humidity               187
Cloudiness             187
Wind Speed             187
Weather Description    187
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [40]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Rikitea,PF,81.21,light rain,-23.1203,-134.9692,
1,Dingle,PH,84.90,scattered clouds,10.9995,122.6711,
2,Puerto Ayora,EC,77.00,few clouds,-0.7393,-90.3518,
5,Vaini,TO,80.60,few clouds,-21.2000,-175.2000,
6,San Cristobal,VE,77.00,scattered clouds,7.7669,-72.2250,
13,Hirara,JP,75.20,scattered clouds,24.8000,125.2833,
14,Gladstone,AU,86.00,light rain,-23.8500,151.2500,
15,Touros,BR,77.00,clear sky,-5.1989,-35.4608,
21,Atuona,PF,79.27,overcast clouds,-9.8000,-139.0333,
22,Harper,LR,78.06,light rain,4.3750,-7.7169,


In [63]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
  
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found...skipping")
   

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [65]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().count()

City                   187
Country                187
Max Temp               187
Weather Description    187
Lat                    187
Lng                    187
Hotel Name             187
dtype: int64

In [68]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [74]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description,</dt><dd>{Weather Description}
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [75]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))